In [1]:
import sys
from pyspark import SparkContext
import json
import time

In [2]:
# task2, input format: task2.py <review_filepath> <output_filepath> <n_partition>
# review_path = sys.argv[1]
# output_path = sys.argv[2]
# n_partition = int(sys.argv[3])
review_path = "./data/test_review.json"
output_path = "./data/task2.json"
n_partition = 4
sc = SparkContext("local[*]",appName="task2").getOrCreate()
review = sc.textFile(review_path).map(lambda x: json.loads(x))
output = {}

23/02/03 17:59:05 WARN Utils: Your hostname, NotredeMacBook-Air.local resolves to a loopback address: 127.0.0.1; using 10.26.112.211 instead (on interface en0)
23/02/03 17:59:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/02/03 17:59:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [56]:
# default
default = {}
s_time = time.time()
review.map(lambda x: [x["business_id"],1]).reduceByKey(lambda x,y: x+y).sortBy(lambda x:[-x[1],x[0]]).take(10)
e_time = time.time()

default["n_partition"] = review.getNumPartitions()
default["n_items"] = review.glom().map(lambda x: len(x)).collect()
# default["n_items"] = review.mapPartitions(lambda x: [len(list(x))]).collect()
default["exe_time"] = e_time-s_time

In [59]:
# customized
customized = {}
s_time = time.time()
review_new = review.map(lambda x: (x['business_id'], 1)).partitionBy(n_partition, lambda x: ord(x[0]))
review_new.reduceByKey(lambda x,y: x+y).sortBy(lambda x:[-x[1],x[0]]).take(10)
e_time = time.time()

customized["n_partition"] = review_new.getNumPartitions()
customized["n_items"] = review_new.glom().map(lambda x: len(x)).collect()
customized["exe_time"] = e_time-s_time

In [70]:
output["default"] = default
output["customized"] = customized
print(output)
with open(output_path,"w") as f:
    json.dump(output,f) 

#export PYSPARK_PYTHON=python3.6                                                                                  
#export JAVA_HOME=/usr/lib/jvm/java-1.8.0-openjdk-amd64  
#/opt/spark/spark-3.1.2-bin-hadoop3.2/bin/spark-submit --executor-memory 4G --driver-memory 4G task2.py
# "../resource/asnlib/publicdata/test_review.json" "./task2.json" 3

{'default': {'n_partition': 2, 'n_items': [13, 17], 'exe_time': 0.12612104415893555}, 'customized': {'n_partition': 3, 'n_items': [10, 7, 13], 'exe_time': 0.1925191879272461}}
